In [2]:
from pytrends.request import TrendReq
from datetime import datetime as dt
import datetime

pytrends_daily = TrendReq(hl='en-US', tz=360)
pytrends_weekly = TrendReq(hl='en-US', tz=360)

In [3]:
#Parameters
#Week starts on Sunday
#Start and end must be within 90 days for daily data

start = '2022-01-02'
end = '2022-04-03'
kw_list = ['bitcoin','ethereum']


end_dt = dt.strptime(end,"%Y-%m-%d")
end_dt_last_yr = dt(end_dt.year - 1, end_dt.month,end_dt.day)
end_last_yr = end_dt_last_yr.strftime("%Y-%m-%d")

tf_daily = start +' '+ end
tf_weekly = end_last_yr + ' ' + end


In [4]:
pytrends_daily.build_payload(kw_list,cat=0,timeframe = tf_daily)
daily_data = pytrends_daily.interest_over_time()
daily_data = daily_data[kw_list]
#daily_data

In [5]:
pytrends_weekly.build_payload(kw_list,cat=0,timeframe = tf_weekly)
weekly_data = pytrends_weekly.interest_over_time()
weekly_data = weekly_data[kw_list]
#weekly_data

In [6]:
for kw in kw_list:
    daily_data[kw + '_wk'] = 0

In [7]:
for index_daily in daily_data.index:
    for index_weekly in weekly_data.index:
        if (index_daily == index_weekly):
            for i in range(7):
                try:
                    daily_data.loc[index_daily]['bitcoin_wk'] = weekly_data.loc[index_daily]['bitcoin']
                    daily_data.loc[index_daily]['ethereum_wk'] = weekly_data.loc[index_daily]['ethereum']
                except:
                    pass
            

In [8]:
daily_data['BTC SVI Adj Factor'] = daily_data['bitcoin_wk']/daily_data['bitcoin']
daily_data['ETH SVI Adj Factor'] = daily_data['ethereum_wk']/daily_data['ethereum']

In [9]:
SVI_adj_factor = daily_data[['BTC SVI Adj Factor','ETH SVI Adj Factor']].copy(deep = True)

In [10]:
for index in weekly_data.index[:-1]:
    for i in range (7):
        try:
            SVI_adj_factor.loc[index + datetime.timedelta(days = i),"BTC SVI Adj Factor"] = SVI_adj_factor.loc[index,"BTC SVI Adj Factor"]  
            SVI_adj_factor.loc[index + datetime.timedelta(days = i),"ETH SVI Adj Factor"] = SVI_adj_factor.loc[index,"ETH SVI Adj Factor"]  
        except:
            pass

In [11]:
daily_data["BTC SVI Adj Factor"] = SVI_adj_factor["BTC SVI Adj Factor"]
daily_data["ETH SVI Adj Factor"] = SVI_adj_factor["ETH SVI Adj Factor"]

In [12]:
daily_data["bitcoin Adj"] = daily_data["bitcoin"]*daily_data["BTC SVI Adj Factor"]
daily_data["ethereum Adj"] = daily_data["ethereum"]*daily_data["ETH SVI Adj Factor"]

In [13]:
daily_data.describe()

,bitcoin,ethereum,bitcoin_wk,ethereum_wk,BTC SVI Adj Factor,ETH SVI Adj Factor,bitcoin Adj,ethereum Adj
count,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,46.891304,9.260870,5.184783,1.173913,0.832151,0.899534,39.011824,8.314152
std,12.760220,2.237883,12.546992,2.861678,0.092200,0.098229,12.155848,2.243286
min,32.000000,5.000000,0.000000,0.000000,0.684932,0.750000,25.000000,5.000000
25%,39.000000,8.000000,0.000000,0.000000,0.800000,0.857143,31.724806,7.000000
50%,44.000000,9.000000,0.000000,0.000000,0.818182,0.875000,37.167421,7.937500
75%,50.250000,10.000000,0.000000,0.000000,0.843750,1.000000,41.533721,9.000000
max,100.000000,18.000000,50.000000,11.000000,1.054054,1.111111,105.405405,18.888889


In [14]:
daily_data["bitcoin Adj"] = daily_data["bitcoin Adj"]*(100/daily_data["bitcoin Adj"].max())

In [15]:
df_btc = daily_data[['bitcoin','bitcoin Adj']].copy(deep=True)
df_eth = daily_data[['ethereum','ethereum Adj']].copy(deep=True)

In [372]:
#df_btc.to_excel('Datasets/btc-googletrends.xlsx')
#df_eth.to_excel('Datasets/eth-googletrends.xlsx')